Going to play with different Neural Networks to try to model all of the data. I don't seem to have saved a varient of the dataframe where all of the stations are present

In [2]:
import polars as pl
from tqdm import tqdm
import os
import pickle
import re
import numpy as np
from collections import defaultdict

In [3]:
# use CSVs in dir
# file_path = f"data/tfl_CSVs/"
directory = f"data/tfl_CSVs/"
downloaded_files = set(os.listdir(directory))
CSVs = downloaded_files
print(f"There are {len(CSVs)} files: {CSVs}")

There are 417 files: {'221JourneyDataExtract01Jul2020-07Jul2020.csv', '152JourneyDataExtract06Mar2019-12Mar2019.csv', '12JourneyDataExtract29Jun2016-05Jul2016.csv', '306JourneyDataExtract23Feb2022-01Mar2022.csv', '366JourneyDataExtract17Apr2023-23Apr2023.csv', '46JourneyDataExtract22Feb2017-28Feb2017.csv', '194JourneyDataExtract25Dec2019-31Dec2019.csv', '343JourneyDataExtract07Nov2022-13Nov2022.csv', '201JourneyDataExtract12Feb2020-18Feb2020.csv', '219JourneyDataExtract17Jun2020-23Jun2020.csv', '227JourneyDataExtract12Aug2020-18Aug2020.csv', '172JourneyDataExtract24Jul2019-30Jul2019.csv', '210JourneyDataExtract15Apr2020-21Apr2020.csv', '364JourneyDataExtract03Apr2023-09Apr2023.csv', '249JourneyDataExtract20Jan2021-26Jan2021.csv', '204JourneyDataExtract04Mar2020-10Mar2020.csv', '69JourneyDataExtract01Aug2017-07Aug2017.csv', '345JourneyDataExtract21Nov2022-27Nov2022.csv', '322JourneyDataExtract15Jun2022-21Jun2022.csv', '352JourneyDataExtract09Jan2023-15Jan2023.csv', '377JourneyDataExtrac

In [4]:
# Regular expression to find 4-digit years in the filenames
year_pattern = r'\d{4}'

# Set to store unique years
years = set()

year_files = defaultdict(list)

# Iterate through the file names and extract years
for file in CSVs:
    years = re.findall(year_pattern, file)
    for year in years:
        year_files[year].append(file)

year_files = dict(year_files)
print(year_files)

year_files = dict(sorted(year_files.items()))

for k, v in year_files.items():
    print(f"for year {k}, there are {len(v)} files: {v}")




{'2020': ['221JourneyDataExtract01Jul2020-07Jul2020.csv', '221JourneyDataExtract01Jul2020-07Jul2020.csv', '201JourneyDataExtract12Feb2020-18Feb2020.csv', '201JourneyDataExtract12Feb2020-18Feb2020.csv', '219JourneyDataExtract17Jun2020-23Jun2020.csv', '219JourneyDataExtract17Jun2020-23Jun2020.csv', '227JourneyDataExtract12Aug2020-18Aug2020.csv', '227JourneyDataExtract12Aug2020-18Aug2020.csv', '210JourneyDataExtract15Apr2020-21Apr2020.csv', '210JourneyDataExtract15Apr2020-21Apr2020.csv', '204JourneyDataExtract04Mar2020-10Mar2020.csv', '204JourneyDataExtract04Mar2020-10Mar2020.csv', '226JourneyDataExtract05Aug2020-11Aug2020.csv', '226JourneyDataExtract05Aug2020-11Aug2020.csv', '243JourneyDataExtract02Dec2020-08Dec2020.csv', '243JourneyDataExtract02Dec2020-08Dec2020.csv', '217JourneyDataExtract03Jun2020-09Jun2020.csv', '217JourneyDataExtract03Jun2020-09Jun2020.csv', '195JourneyDataExtract01Jan2020-07Jan2020.csv', '195JourneyDataExtract01Jan2020-07Jan2020.csv', '231JourneyDataExtract09Sep202

In [5]:
# CSVs = CSVs[:50]

In [6]:
# Initialize an empty Polars DataFrame
pickle_path = "pickle/CSV_to_df/"
all_cols = {}

CSVs_to_df = False

if CSVs_to_df:
    for k, v in year_files.items():
        merged_df = pl.DataFrame()
        cols = []

        progress_bar = tqdm(v, desc=f"Processing files from {k}")

        for idx, csv in enumerate(progress_bar):
            file_path = f"{directory}{csv}"
            # print(f"CSV: {csv}\n file_path: {file_path}")
            df = pl.read_csv(file_path, skip_rows_after_header=1, ignore_errors=True)
            for c in df.columns:
                if c in all_cols:
                    all_cols[c] += 1
                else:
                    all_cols[c] = 1

            # Rename columns and keep only 'Rental ID', 'Start Date', 'Start Station Name', 'End Date', 'End Station Name'
            renaming_dict = {
                'Number': 'Rental ID',
                'rental ID': 'Rental ID',
                'Rental Id': 'Rental ID',
                'Start date': 'Start Date',
                'StartStation Name': 'Start Station Name',
                # 'StartStation Id' : 'Start Station ID',
                # 'Start Station Id' : 'Start Station ID',
                # 'Start station number': 'Start Station ID',
                # 'StartStation Logical Terminal': 'Start Station ID',
                # 'EndStation Id' : 'End Station ID',
                # 'End Station Id' : 'End Station ID',
                # 'End station number': 'End Station ID',
                # 'EndStation Logical Terminal' : 'End Station ID',
                # 'endStationPriority_id' : 'End Station ID',
                'Start station': 'Start Station Name',
                'End date': 'End Date',
                'EndStation Name': 'End Station Name',
                'End station': 'End Station Name'
            }

            for old_col, new_col in renaming_dict.items():
                if old_col in df.columns:
                    df = df.rename({old_col: new_col})
            
            cols_to_keep = ['Rental ID', 'Start Date', 'Start Station Name', 'End Date', 'End Station Name']
            df = df.select(cols_to_keep)

            # Convert date columns
            df = df.with_columns([
                pl.col('Start Date').str.strptime(pl.Datetime, strict=False),
                pl.col('End Date').str.strptime(pl.Datetime, strict=False),
                pl.col('Start Station Name').cast(pl.Utf8),
                pl.col('End Station Name').cast(pl.Utf8)
            ])

            # Filter rows (if needed)
            # df = df.filter((pl.col("Start Station Name") == "Waterloo Station 3, Waterloo") |
            #                (pl.col("End Station Name") == "Waterloo Station 3, Waterloo"))
            
            # Concatenate DataFrames
            merged_df = pl.concat([merged_df, df])

            progress_bar.set_description(
                f"Creating merged_df for {k}. merged_df is {merged_df.estimated_size()/(1024*1024)}MB large"
            )
        pickle_dir = f"{pickle_path}/CSV_to_df_{k}"
        # save to pickle
        with open(pickle_dir, 'wb') as f:
            pickle.dump(merged_df, f)

In [7]:
# experiment to look at column values in the input CSVs
if CSVs_to_df:
    mapping_dict = {
                    'Number': 'Rental ID',
                    'rental ID': 'Rental ID',
                    'Rental Id': 'Rental ID',
                    'Start date': 'Start Date',
                    'StartStation Name': 'Start Station Name',
                    'StartStation Id' : 'Start Station ID',
                    'Start Station Id' : 'Start Station ID',
                    'Start station number': 'Start Station ID',
                    'StartStation Logical Terminal': 'Start Station ID',
                    'EndStation Id' : 'End Station ID',
                    'End Station Id' : 'End Station ID',
                    'End station number': 'End Station ID',
                    'EndStation Logical Terminal' : 'End Station ID',
                    'endStationPriority_id' : 'End Station ID',
                    'Start station': 'Start Station Name',
                    'End date': 'End Date',
                    'EndStation Name': 'End Station Name',
                    'End station': 'End Station Name'
                }

    def apply_dict_mapping(mapping, d):
        for k,v in mapping_dict.items():
            d[v] = d.get(k,0) + d.get(v,0)
            if k in d:
                del d[k]
        return d

    in_dict = all_cols

    out_dict = apply_dict_mapping(mapping_dict, in_dict)

    for c, n in out_dict.items():
        print(f"{c}: {n}")

In [8]:
#Generate Station to ID mapping
generate_station_to_ID_mapping = False
station_names_pickle_path = f"{pickle_path}/station_names"

if generate_station_to_ID_mapping:
    station_df = pl.Series(name="Station Names", values=[], dtype=pl.Utf8)
    for year in year_files.keys():
        #load from pickle
        pickle_dir = f"{pickle_path}/CSV_to_df_{year}"
        with open(pickle_dir, 'rb') as f:
            df = pickle.load(f)
            df_start = df['Start Station Name'].unique()
            df_end = df['End Station Name'].unique()
            station_df = pl.concat([station_df, df_start, df_end])
            
    station_df = station_df.drop_nans().drop_nulls()

    station_mapping = {s: i for i, s in enumerate(sorted(station_df.unique().to_list()))}

    mapping_df = pl.DataFrame({
        "Station Name": list(station_mapping.keys()),
        "ID": list(station_mapping.values())
    })

    
    
    
    with open(station_names_pickle_path, 'wb') as f:
            pickle.dump(mapping_df, f)
else:
     with open(station_names_pickle_path, 'rb') as f:
            mapping_df = pickle.load(f)

print(mapping_df.head())

shape: (5, 2)
┌─────────────────────────────────┬─────┐
│ Station Name                    ┆ ID  │
│ ---                             ┆ --- │
│ str                             ┆ i64 │
╞═════════════════════════════════╪═════╡
│ Abbey Orchard Street, Westmins… ┆ 0   │
│ Abbotsbury Road, Holland Park   ┆ 1   │
│ Aberdeen Place, St. John's Woo… ┆ 2   │
│ Aberfeldy Street, Poplar        ┆ 3   │
│ Abingdon Green, Westminster     ┆ 4   │
└─────────────────────────────────┴─────┘


In [9]:
# Generate yearly flows
generate_yearly_flows = False
flows_pickle_path = "pickle/flows"

if generate_yearly_flows:
    for year in tqdm(year_files.keys()):
        #load from pickle
        pickle_dir = f"{pickle_path}/CSV_to_df_{year}"
        with open(pickle_dir, 'rb') as f:
            df = pickle.load(f)

        df = df.join(mapping_df.rename({"Station Name": "Start Station Name", "ID": "Start ID"}), on="Start Station Name", how="left")
        df = df.join(mapping_df.rename({"Station Name": "End Station Name", "ID": "End ID"}), on="End Station Name", how="left")

        df_start = df["Rental ID", "Start Date", "Start ID"].rename({"Start Date": "Date", "Start ID": "ID"}).with_columns(pl.lit(-1).alias("flow"))
        df_end = df["Rental ID", "End Date", "End ID"].rename({"End Date": "Date", "End ID": "ID"}).with_columns(pl.lit(1).alias("flow"))
        df_flows = pl.concat([df_start, df_end]).sort("Date")

        df_flow_filt = df_flows.filter(df_flows["Date"].dt.year()==year)

        with open(f"{flows_pickle_path}/flow_{year}", 'wb') as f:
            pickle.dump(df_flow_filt, f)

In [10]:
#Aggregate flows
aggregate_flows = False
aggregate_time_period = "15m"
aggreagte_time_col_name = "15minWindow"
agg_pickle_path = f"pickle/aggregated/{aggregate_time_period}"

def aggregate_by_time(df, time_str="15m", time_str_name='15minWindow'):
    df_agg = (
        df
        .with_columns(pl.col("Date").dt.truncate(time_str).alias(time_str_name))
        .group_by(["ID", time_str_name])
        .agg(pl.col("flow").sum().alias("total_flow"))
        )
    return df_agg
# fifteen_min_agg = aggregate_by_time(df_flows)
# print(fifteen_min_agg.describe())
# print(f"merged_df is {fifteen_min_agg.estimated_size()/(1024*1024)}MB large")

if aggregate_flows:
    for year in tqdm(year_files.keys()):
        #load from pickle
        pickle_dir = f"{flows_pickle_path}/flow_{year}"
        with open(pickle_dir, 'rb') as f:
            df = pickle.load(f)
        df_agg = aggregate_by_time(df, time_str=aggregate_time_period, time_str_name=aggreagte_time_col_name)

        with open(f"{agg_pickle_path}/agg_{year}", 'wb') as f:
            pickle.dump(df_agg, f)

In [11]:
#Concatenate aggregated flows together
concatenate_agg_flows=False
concat_pickle_path = f"pickle/concat/concat_df_{aggregate_time_period}"

if concatenate_agg_flows:
    merged_df = None
    for year in tqdm(year_files.keys()):
        with open(f"{agg_pickle_path}/agg_{year}", 'rb') as f:
            df = pickle.load(f)
            # print(f"DataFrame {year} columns: {df.columns} with types: {df.dtypes}")

        # Concatenate DataFrames
        if merged_df is None:
             merged_df = df
        else:
            merged_df = pl.concat([merged_df, df], how='vertical')
    with open(f"{concat_pickle_path}", 'wb') as f:
            pickle.dump(merged_df, f)

In [12]:
with open(f"{concat_pickle_path}", 'rb') as f:
    merged_df = pickle.load(f)

In [13]:
merged_df.describe(), merged_df.head(), merged_df.tail()

(shape: (9, 4)
 ┌────────────┬─────────────┬────────────────────────────┬─────────────┐
 │ statistic  ┆ ID          ┆ 15minWindow                ┆ total_flow  │
 │ ---        ┆ ---         ┆ ---                        ┆ ---         │
 │ str        ┆ f64         ┆ str                        ┆ f64         │
 ╞════════════╪═════════════╪════════════════════════════╪═════════════╡
 │ count      ┆ 7.6451699e7 ┆ 76452792                   ┆ 7.6452792e7 │
 │ null_count ┆ 1093.0      ┆ 0                          ┆ 0.0         │
 │ mean       ┆ 447.14965   ┆ 2020-04-14 20:25:31.945749 ┆ -0.004108   │
 │ std        ┆ 261.682595  ┆ null                       ┆ 3.855214    │
 │ min        ┆ 0.0         ┆ 2016-02-07 00:00:00        ┆ -158.0      │
 │ 25%        ┆ 223.0       ┆ 2018-05-01 09:15:00        ┆ -2.0        │
 │ 50%        ┆ 442.0       ┆ 2020-05-28 15:45:00        ┆ 0.0         │
 │ 75%        ┆ 674.0       ┆ 2022-04-24 20:00:00        ┆ 2.0         │
 │ max        ┆ 899.0       ┆ 2024-0

In [14]:
null_counts = merged_df.null_count()
print(null_counts)

merged_df = merged_df.drop_nulls(subset=["ID"])

shape: (1, 3)
┌──────┬─────────────┬────────────┐
│ ID   ┆ 15minWindow ┆ total_flow │
│ ---  ┆ ---         ┆ ---        │
│ u32  ┆ u32         ┆ u32        │
╞══════╪═════════════╪════════════╡
│ 1093 ┆ 0           ┆ 0          │
└──────┴─────────────┴────────────┘


In [15]:
len(merged_df['ID'].unique()), merged_df['15minWindow'].max() - merged_df['15minWindow'].min()

(900, datetime.timedelta(days=3018, seconds=45000))

901 stations, 3018 days -> 289728 15min intervals so 261million elemements in array of which 76million will have a value

In [16]:
seconds_in_window = 60*15
start_timestamp = merged_df['15minWindow'].min()
time_range = merged_df['15minWindow'].max() - start_timestamp
N_time_intervals=int(time_range.total_seconds()/seconds_in_window)+1
N_ids = len(merged_df['ID'].unique())

print(f"Array dims: ({N_ids}, {N_time_intervals})")

data_array = np.zeros((N_ids, N_time_intervals), dtype=np.int32)

Array dims: (900, 289779)


In [17]:
for row in tqdm(merged_df.iter_rows(), total=merged_df.shape[0]):
    id_idx = row[merged_df.columns.index("ID")]
    timestamp = row[merged_df.columns.index(aggreagte_time_col_name)]
    total_flow = row[merged_df.columns.index("total_flow")]

    time_idx = int((timestamp-start_timestamp).total_seconds()/seconds_in_window)

 # Ensure time_idx is within bounds
    if 0 <= time_idx < data_array.shape[1]:  # Check time index bounds
        data_array[id_idx, time_idx] = total_flow
    else:
        print(f"Out of bounds: ID={id_idx}, Time Index={time_idx}, Timestamp={timestamp}")

with open(f"pickle/numpy_flows", 'wb') as f:
    pickle.dump(data_array, f)




100%|██████████| 76451699/76451699 [04:02<00:00, 315621.28it/s]
